In [25]:
%matplotlib inline
import nltk
import pandas as pd
import matplotlib.pyplot as plt
pd.set_option('display.max_colwidth', 100)

In [14]:
import xml.etree.ElementTree as ET
train_data = ET.parse('line-train.xml')

In [ ]:
import string
string.punctuation

In [15]:
test =  ET.parse('line-test.xml')

In [16]:
def process_children(element):
    text = ''
    for child in element.getchildren():
        try:
            text = text + ' ' + child.text
        except TypeError:
            pass
    return text

In [17]:
from copy import copy
def get_context_list(xmlobject):
    data = []
    dict_item = {}
    iterator = xmlobject.iter()
    for element in iterator:
        if element.tag == 'context':
            dict_item['context'] = process_children(element)
            data.append(copy(dict_item))
            dict_item = {}
            continue
        elif element.tag == 'answer':
            dict_item['senseid'] = element.attrib.get('senseid')
            dict_item['element'] = ET.tostring(element).decode("utf-8").strip('\n')
            continue
    return data

In [18]:
data = get_context_list(train_data)

In [19]:
df_train = pd.DataFrame(data, columns=["element", 'senseid', 'context'])

In [20]:
def get_context_list_test(xmlobject):
    data = []
    data_item = {}
    iterator = xmlobject.iter()
    for element in iterator:
        if element.tag == 'instance':
            data_item['id'] = element.attrib.get('id')
            data_item['context'] = process_children(element.getchildren()[0])
            data.append(copy(data_item))
    return data

In [21]:
data = get_context_list_test(test)

In [22]:
df_test = pd.DataFrame(data, columns=['id', 'context'])

In [23]:
df_test['tokens'] = df_test['context'].apply(nltk.word_tokenize)

In [27]:
stopwords = nltk.corpus.stopwords.words('english')
remove_punct = lambda text: ''.join((char for char in text if char not in string.punctuation))
remove_stopwords = lambda token_list: [word for word in token_list if word not in stopwords]

In [158]:
from sklearn.feature_extraction.text import CountVectorizer

In [160]:
vectorizer = CountVectorizer()

In [161]:
X = vectorizer.fit_transform(df['context'])

In [163]:
print(len(vectorizer.get_feature_names()))

3303


In [167]:
X.shape

(374, 3303)

In [169]:
df.shape

(374, 4)